<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/Federated_Learning%20/Notebooks/DP_FL_Default_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
#Importa Pytorch, Flower e Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [134]:
#Importa as bibliotecas necessárias
from collections import OrderedDict
from typing import Dict, Optional, List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
import opacus
from opacus import PrivacyEngine
from torch.optim.adam import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [135]:
#Define o dispositivo de treinamento
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__}, Flower{fl.__version__} and Opacus{opacus.__version__ }")

Training on cpu using Pytorch2.0.1+cu118, Flower1.4.0 and Opacus1.4.0


In [136]:
# Define os valores dos hiperparâmetros de treinamento
BATCH_SIZE = 256
NUM_CLIENTS= 10
optim_lr=0.001
local_epochs=1
num_rounds=2
noise_multiplier=0.1
max_grad_norm=1.0

In [137]:
#Carrega os dados
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    # Realiza a partição do dataset CIFAR10 em n clientes
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10 # 10% do conjunto para testes
        len_train = len(ds) - len_val
        lengths =[len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,num_workers=0))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False,num_workers=0))
        testloader = DataLoader(testset,batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
        num_examples = {"ds_train": len(ds_train)*BATCH_SIZE, "ds_val": len(ds_val)*BATCH_SIZE}

    return trainloaders, valloaders, testloader,num_examples
    
trainloaders, valloaders, testloader,num_examples = load_data()

Files already downloaded and verified
Files already downloaded and verified


In [138]:
# Define o Modelo de Rede Neural
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [139]:
# Define as funções de treino e teste
def train(net, trainloader,optimizer,privacy_engine, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [140]:
# Define as funções de atualização de parâmetros do modelo de rede neural
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [141]:
# Define a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, num_example,optimizer,privacy_engine)-> None:
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.num_example = num_example
        self.optimizer = optimizer
        self.privacy_engine = privacy_engine

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,self.optimizer,self.privacy_engine,epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader = trainloaders[int(cid)]
        valloader = valloaders[int(cid)]
        num_example = num_examples
        optimizer = torch.optim.Adam(net.parameters(),lr=optim_lr)
        privacy_engine = PrivacyEngine()
        net, optimizer, trainloader = privacy_engine.make_private(
        module=net,
        optimizer=optimizer,
        data_loader=trainloader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm, )
        return FlowerClient(cid, net, trainloader, valloader, num_example,optimizer,privacy_engine)

In [142]:
# Define a função de agregação do aorendizado federado
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [143]:
# Inicializa o treino federado
# Cria a função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
# Define o dispositivo a ser utilizado pelos clientes
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)
     

INFO flwr 2023-06-11 22:49:50,349 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)


(launch_and_evaluate pid=26500) [Client 1] evaluate, config: {} [repeated 4x across cluster]


2023-06-11 22:49:57,520	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-11 22:49:59,327 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3903876710.0, 'memory': 7807753422.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3903876710.0, 'memory': 7807753422.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-11 22:49:59,330 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-11 22:49:59,338 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=30899) 2023-06-11 22:50:04.163162: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO flwr 2023-06-11 22:50:08,998 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one r

(launch_and_get_parameters pid=30899) [Client 8] get_parameters


(launch_and_get_parameters pid=30899) /usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
(launch_and_get_parameters pid=30899)   warnings.warn(


...

(launch_and_fit pid=30899) [Client 9] fit, config: {}
(launch_and_evaluate pid=30898) [Client 9] evaluate, config: {} [repeated 3x across cluster]
(launch_and_fit pid=30899) [Client 1] fit, config: {} [repeated 2x across cluster]
(launch_and_fit pid=30898) [Client 4] fit, config: {} [repeated 2x across cluster]
(launch_and_fit pid=30898) [Client 8] fit, config: {} [repeated 2x across cluster]
(launch_and_fit pid=30898) [Client 2] fit, config: {} [repeated 2x across cluster]


DEBUG flwr 2023-06-11 22:51:55,382 | server.py:232 | fit_round 2 received 10 results and 0 failures
DEBUG:flwr:fit_round 2 received 10 results and 0 failures
DEBUG flwr 2023-06-11 22:51:55,407 | server.py:168 | evaluate_round 2: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 2: strategy sampled 10 clients (out of 10)


(launch_and_evaluate pid=30899) [Client 7] evaluate, config: {}
(launch_and_fit pid=30899) [Client 7] evaluate, config: {}
(launch_and_evaluate pid=30899) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=30898) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=30899) [Client 3] evaluate, config: {}


DEBUG flwr 2023-06-11 22:52:09,157 | server.py:182 | evaluate_round 2 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 10 results and 0 failures
INFO flwr 2023-06-11 22:52:09,162 | server.py:147 | FL finished in 120.1504263880006
INFO:flwr:FL finished in 120.1504263880006
INFO flwr 2023-06-11 22:52:09,164 | app.py:218 | app_fit: losses_distributed [(1, 0.008912516450881956), (2, 0.008201720404624939)]
INFO:flwr:app_fit: losses_distributed [(1, 0.008912516450881956), (2, 0.008201720404624939)]
INFO flwr 2023-06-11 22:52:09,166 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-11 22:52:09,167 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.2048), (2, 0.2674)]}
INFO:flwr:app_fit: metrics_distributed {'accuracy': [(1, 0.2048), (2, 0.2674)]}
INFO flwr 2023-06-11 22:52:09,169 | app.py:221 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-06-11 22:52:0

History (loss, distributed):
	round 1: 0.008912516450881956
	round 2: 0.008201720404624939
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.2048), (2, 0.2674)]}